In [1]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import re
import sys
from collections import defaultdict, deque
from typing import List
import re
from typing import Any, List
import ipydagred3
import networkx as nx
import numpy as np
import openai
from tqdm import tqdm


from P3_TOT_utils import *

#os.environ["http_proxy"] = "http://localhost:7890"
#os.environ["https_proxy"] = "http://localhost:7890"
setOpenAi(keyid = 0)

GPT_MODEL = "gpt-4-turbo"
GPT_MODEL = "gpt-4o-mini"
GPT_MODEL = "gpt-3.5-turbo"



In [2]:
response = openai.ChatCompletion.create(
    model = GPT_MODEL,
    messages = [{"role":'user', "content": "How to make a cake"}],
    logprobs= True,
    top_logprobs = 2
    
    )
response

<OpenAIObject chat.completion id=chatcmpl-A2qalyeeAKxViL07TS2ok5KiBXGRW at 0x10de848c0> JSON: {
  "id": "chatcmpl-A2qalyeeAKxViL07TS2ok5KiBXGRW",
  "object": "chat.completion",
  "created": 1725242183,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1. Preheat your oven to the temperature specified in your recipe.\n\n2. Grease and flour your cake pans to prevent sticking.\n\n3. In a mixing bowl, cream together butter and sugar until light and fluffy.\n\n4. Beat in eggs one at a time, mixing well after each addition.\n\n5. In a separate bowl, combine flour, baking powder, and salt. Gradually add this dry mixture to the wet ingredients, alternating with milk.\n\n6. Stir in any additional flavorings or mix-ins, such as vanilla extract or chocolate chips.\n\n7. Pour the batter into your prepared cake pans, smoothing the tops with a spatula.\n\n8. Bake in the preheated oven until a toothpick inserted

In [3]:
n = -0.00055124465

In [4]:
np.round(np.exp(n)*100,2)

99.94

In [6]:
puzzles = []


with open("/Users/natehu/Desktop/Tsinghua Research/Tsinghua TaDe/TaDe/dataset_gen_using_cot/p3_100_test_questions.json", "r") as f:
    puzzles = json.load(f)

In [7]:
puzzles[0]

{'name': 'ReverseCat:3',
 'sat': "def sat(s: str, substrings=['thacovatukoliva', 'maquyfezisothizyp', 'ka', 'benegiquememif']):\n    return all(sub in s and sub[::-1] in s for sub in substrings)",
 'ans_type': 'str',
 'sol_header': "def sol(substrings=['thacovatukoliva', 'maquyfezisothizyp', 'ka', 'benegiquememif']):",
 'sol_docstring': '    """\n    Find a string that contains all the substrings reversed and forward\n    """',
 'sol_bodies': ['    return "".join(substrings + [s[::-1] for s in substrings])'],
 'module': 'basic.py',
 'notes': '',
 'weight': 1.0}

In [21]:
id = 3

question = puzzles[id]["sat"]


In [22]:
decompose_steps = decompose_sql(question)
steps, steps_dict = convert_steps_to_format(decompose_steps)
num_steps = len(steps)
print(steps)
print(steps_dict)  # 只是加了一个问题的编号而已.

[{'stepId': 1, 'step': ' List the elements from the provided list `li`. '}, {'stepId': 2, 'step': ' Create a permutation of the list `li`. '}, {'stepId': 3, 'step': ' Select the first element in the permutation to match the minimum value of the remaining list. '}, {'stepId': 4, 'step': ' For the second element, choose it to match the maximum value of the remaining list. '}, {'stepId': 5, 'step': ' Repeat alternating selection of minimum and maximum values from the remaining elements of the list for each subsequent position. '}, {'stepId': 6, 'step': ' Check if the final permutation satisfies the conditions: Elements at odd indices should be the minimum of the remaining elements, and elements at even indices should be the maximum of the remaining elements. '}, {'stepId': 7, 'step': ' If Step 6 fails, return to Step 2 and choose a different permutation until the conditions are fulfilled. Once Step 6 passes, the permutation is the solution. '}]
{1: ' List the elements from the provided li

In [23]:
len(steps)

7

In [11]:
# TOT 求解
N = 3  # 每个子问题进行N次proposal
M = 2  # 通过评估选出M个最好的proposal

solution = []

for i in range(num_steps):
    subtask = steps[i]
    sys_q = f"""
    There is a programming puzzle. Your final goal is to find an input for the function that will return True.
    
    Here is the problem:\n{question}

    Here, I have broken this puzzle down into a series of smaller problems. I will assign you sub-problems one by one, and provide the results of the previous sub-problems as a reference for your reasoning.
    
    Please solve the problem and respond according to the logic and results of the previous sub-problems.

"""  

    subask = f"""\nThe sub-steps to be solved now is:: {subtask}
Based on the information above, please provide a concise and clear answer"""
    
    if len(solution)==0:
        # 第一个子问题
        query = subask
        Q = [{'role':'system', 'content':sys_q},
            {'role':'user', 'content':query},]
        for n in range(N):  # 一个子问题提问N次,获取N个解
            result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
            eval_Q = Q + [{'role':'assistant', 'content':result}]
            eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
            score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
            score = int(score)
            
            solution.append((score, [result]))  # 维护一整条推理路径
            
        solution = sorted(solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝
    else:
        temp_solution = []
        for m in range(M):  # 因为剪枝动态维护M个推理路径
            answersSoFar = f"""\nSo far, the answers to the preceding sub-instructions are as follows: The format is Sub-problem-Id: xxx; Sub-problem: xxx; Answer: xxx."""
            for index, value in enumerate(solution[m][1]):
                try:
                    answersSoFar += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""
                except:
                    print('warning')
                    print(index)
                    print(len(solution[m][1]))
                    print(len(steps))
                    sys.exit(0)
            query = answersSoFar+subask
            Q = [{'role':'system', 'content':sys_q},
                 {'role':'user', 'content':query},]
            for n in range(N):  # 一个子问题提问N次,获取N个解
                result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
                eval_Q = Q + [{'role':'assistant', 'content':result}]
                eval_Q = eval_Q + [{'role':'user', 'content':"Please provide a confidence rating for the accuracy of this solution, on a scale from 1 to 5. Only output the number."}]
                score = askChatGPT(eval_Q, model=GPT_MODEL, temperature=1)
                score = int(score)
                
                temp_solution.append((solution[m][0]+score, solution[m][1]+[result]))  # 路径score累加
        
        # print(len(temp_solution))  # 此时temp_solution中应该有M*N种推理路径
        solution = sorted(temp_solution, key=lambda x: x[0])
        solution = solution[:M]  # 剪枝 M*N->M

print(len(solution))
printSeq(solution)
# 用额外的一次query再问一下最终的答案


2
(35, ["To split the string 's' into individual words, we can use the `split()` method in Python. This method splits a string into a list based on a specified separator, which is typically a space ' ' in this case. The code `s.split()` will split the string 's' into individual words based on spaces.", 'Sub-problem-Id: 2; Sub-problem: For each word, count the number of consonants.; Answer: To count the number of consonants in each word, iterate through each character in the word and increment a counter if the character is a consonant (not in "aeiou").', "Sub-problem-Id: 3; Sub-problem: Identify words with exactly 2 consonants.; Answer: To identify words with exactly 2 consonants in the given string, you should check if the number of consonants in each word is equal to 2 as per the previous step's logic.", "Sub-problem-Id: 4; Sub-problem: List out the words identified in the previous step in the order they appear.; Answer: In the previous step, we identified words with exactly 2 consona

In [ ]:
solution

[(34,
  ["To create a list of words that will satisfy the condition for the input parameter 'words', we need words where the number of consonants is equal to the specified value 'n'. For example, if n=2, a potential list of words could be ['textitextomyc', 'regolathagychiby'].",
   "Sub-problem-Id: 1; Sub-problem: Split the string 's' into individual words.; Answer: The string 's' can be split into individual words by using the split() method in Python.",
   'Sub-problem-Id: 3; Sub-problem: Count the number of consonants in each word.; Answer: To count the number of consonants in each word, iterate over each character in the word, convert it to lowercase, and increment a counter if the character is not a vowel (i.e., not in "aeiou").',
   "Sub-problem-Id: 4; Sub-problem: Check if the number of consonants in a word matches the value of 'n.'; Answer: To check if the number of consonants in a word matches the value of 'n', compare the counted number of consonants in the word to the specif

In [66]:
user_q = f"""There is a natural language command representing a sequence of actions:\n{question}

I have broken this action down into a series of smaller pieces and each sub-problem is solved.
The sub-problems and their corresponding answers are as follows. """

for index, value in enumerate(solution[0][1]):  # 这里仅仅使用了最终得分最高的1条路径来总结得到final answer
    user_q += f"""\nSub-problem-Id: {index}; Sub-problem: {steps[index]}; Answer: {value}."""

user_q += "\n Given the results from the sub problems and their answer, so what is the final answer? (Try to concatenate some of the sub-problems  answers to get the final answer.)"

final_Q = [{'role':'user', 'content':user_q}]


print(final_Q)

[{'role': 'user', 'content': 'There is a natural language command representing a sequence of actions:\ndef sat(words: List[str], s="thigafamyhuchykikoxe limyb wy textitextomyc regolathagychiby pep", n=2):\n    i = 0\n    for w in s.split():\n        num_consonants = 0\n        for c in w.lower():\n            if c not in "aeiou":\n                num_consonants += 1\n        if num_consonants == n:\n            if words[i] != w:\n                return False\n            i += 1\n    return i == len(words)\n\nI have broken this action down into a series of smaller pieces and each sub-problem is solved.\nThe sub-problems and their corresponding answers are as follows. \nSub-problem-Id: 0; Sub-problem: {\'stepId\': 1, \'step\': "Create a list of words that will satisfy the condition for the input parameter \'words\'."}; Answer: To create a list of words that will satisfy the condition for the input parameter \'words\', we need words where the number of consonants is equal to the specified

In [67]:
s_result = askChatGPT(final_Q, model=GPT_MODEL, temperature=1)

total tokens: 1192


In [68]:
final_Q.append({'role':'assistant', 'content':s_result})
final_Q.append({'role':'user', 'content':f"""Now that all the sub-tasks have been completed, so what is the correct input?
Please only give the input in the format of a string and just give the answer without any additional explanation or clarification, no prefix or suffix.

For example if the input should be x = 5, then you should only give the answer as 5 and not x = 5.
For example, if the the input is list = [1,2,3], then you should only give the answer as [1,2,3] and not list = [1,2,3].
"""})
finalResult = askChatGPT(final_Q, model=GPT_MODEL, temperature=1)

total tokens: 1334


In [74]:
final_Q

[{'role': 'user',
  'content': 'There is a natural language command representing a sequence of actions:\ndef sat(words: List[str], s="thigafamyhuchykikoxe limyb wy textitextomyc regolathagychiby pep", n=2):\n    i = 0\n    for w in s.split():\n        num_consonants = 0\n        for c in w.lower():\n            if c not in "aeiou":\n                num_consonants += 1\n        if num_consonants == n:\n            if words[i] != w:\n                return False\n            i += 1\n    return i == len(words)\n\nI have broken this action down into a series of smaller pieces and each sub-problem is solved.\nThe sub-problems and their corresponding answers are as follows. \nSub-problem-Id: 0; Sub-problem: {\'stepId\': 1, \'step\': "Create a list of words that will satisfy the condition for the input parameter \'words\'."}; Answer: To create a list of words that will satisfy the condition for the input parameter \'words\', we need words where the number of consonants is equal to the specifi

In [69]:
finalResult

"['textitextomyc', 'regolathagychiby']"

In [70]:
def remove_quotes(s: str) -> str:
    # 去除两边的引号
    if s.startswith('"') and s.endswith('"'):
        return s[1:-1]
    elif s.startswith("'") and s.endswith("'"):
        return s[1:-1]
    else:
        return s
    

In [71]:
from typing import Any, List
type_dict = {
    'str': str,
    'int': int,
    'float': float,
    'bool': bool,
    'List[int]': List[int],
    'List[str]': List[str],
    'List[float]': List[float],
    'List[bool]': List[bool],
    'List[List[int]]': List[List[int]],
    'List[List[float]]': List[List[float]],
    'List[List[str]]': List[List[str]],
    'List[List[List[int]]]': List[List[List[int]]]
}
def convert_to_type(type_str: str, value_str: str) -> Any:
    """
    根据给定的类型字符串和待转换的字符串,将其转换为相应的Python类型
    """
    if type_str in type_dict:
        python_type = type_dict[type_str]
        if type_str.startswith('List'):
            # 处理列表类型
            return eval(value_str)
        else:
            # 处理基本类型
            return python_type(value_str)
    else:
        raise ValueError(f"不支持的类型: {type_str}")



In [72]:
puzzles[id]

{'name': 'ConsonantFilter:4',
 'sat': 'def sat(words: List[str], s="thigafamyhuchykikoxe limyb wy textitextomyc regolathagychiby pep", n=2):\n    i = 0\n    for w in s.split():\n        num_consonants = 0\n        for c in w.lower():\n            if c not in "aeiou":\n                num_consonants += 1\n        if num_consonants == n:\n            if words[i] != w:\n                return False\n            i += 1\n    return i == len(words)',
 'ans_type': 'List[str]',
 'sol_header': 'def sol(s="thigafamyhuchykikoxe limyb wy textitextomyc regolathagychiby pep", n=2):',
 'sol_docstring': '    """Find all words in the string with n consonants\n\n    Sample input:\n    s="An eye for an I", n=1\n    Sample output:\n    ["An", "eye", "an"]\n    """',
 'sol_bodies': ['    return [w for w in s.split() if sum(c.lower() not in "aeiou" for c in w) == n]'],
 'module': 'human_eval.py',
 'notes': 'Inspired by [HumanEval](https://github.com/openai/human-eval) \\#117',
 'weight': 1.0}

In [73]:
exec(question)
# 根据ans_type_str将结果转换为相应的类型
print('final Result')
print(finalResult)
print('should answer type')
print(puzzles[id]['ans_type'])
converted_result = convert_to_type(puzzles[id]['ans_type'], finalResult)
# print('converted_result:\n')
# print(converted_result)
# print('converted_result type') 
# print(type(converted_result))

result = sat(converted_result)
# print("函数运行结果为：", result)
if result == True:
    
    print('success')
else:
    
    print('fail')

final Result
['textitextomyc', 'regolathagychiby']
should answer type
List[str]
fail
